# 응용통계학 과제
> 다중공선성

- toc:true
- branch: master
- badges: true
- comments: false
- author: 최서연
- categories: [Applied Statistics, 다중공선성]

In [52]:
#hide
options(jupyter.plot_scale=4)
options(repr.plot.width=8,repr.plot.height=4,repr.plot.res=500)
#options(jupyter.rich_display=FALSE)
#options(max.print=1000)


다중공선성이 존재하는 상황을 가정하고 

 

다중공선성을 어느 정도 제거한 모형 (M1)과 다중공선성이 내재되어 있는 모형 (M2) 을 고려하여

 

두 모형의 예측력을 모의실험을 통해 비교하여라,

 

단, 실험은 여러 번 반복하여 평균적인 결과를  report하되 설명변수의 개수는 3개 이상으로 설정하여라.

 

이미 존재하는 문서들을 참고하거나 재현해도 무방함.

 

(첨부된 문서 참고)

In [131]:
Nrep = 200

n = 1000

hatb1 <- hatb2 <- c()
te1 <- te2 <- c()

for (k in 1:Nrep)
{
x1 = runif(n,-3,3)
x2 = x1 + rnorm(n,0,0.01)
#X = model.matrix(~ x1 + x2)
#solve(t(X)%*%X)
y = 2 + x1 + 2*x2 + rnorm(n)

ind = sample(1:n,500)
tx1 = x1[ind]
tx2 = x2[ind]
test_x1 = x1[-ind]
test_x2 = x2[-ind]
ty  = y[ind]
test_y = y[-ind]

fit1 = lm(ty~tx1+tx2)
fit2 = lm(ty~tx1)

hatb1[k] = fit1$coefficients[2]
hatb2[k] = fit2$coefficients[2]

te1[k] = mean((test_y - predict(fit1,newdata=data.frame(test_x1,test_x2)))^2)
te2[k] = mean((test_y - predict(fit2,newdata=data.frame(test_x1)))^2)
#summary(lm(y~x1+x2))
#print(k)
}

c(mean(te1),mean(te2))
c(mean((hatb1-1)^2),mean((hatb2-2)^2))

[1] 54.98598 54.98444

[1] 21.033269  1.000548

In [176]:
mean(hatb1)

[1] 1.399897

---

In [165]:
library(regclass)

---

## 시도 1

In [149]:
x1 = runif(100)
x2 = x1 + rnorm(100,0,0.01)
x3 = x1 + rnorm(100,0,0.1)
y1=3 + x1 + rnorm(100)
y2=3 + x1 + 1.5*x2 + 3.5*x3 + rnorm(100)

In [150]:
cor(x1,x2)
cor(x1,x3)

[1] 0.9994958

[1] 0.9396317

높은 상관계수 확인

In [169]:
M1 = lm(y1~x1)

In [170]:
M2 = lm(y2~x1+x2+x3)

In [177]:
VIF(M2)

x1          x2          x3 
1029.922085 1003.549547    8.639639

VIF 10 넘는 모습, x3은 10이 안 넘었다.

In [178]:
M1$coefficients

(Intercept)          x1 
   2.744495    1.386077

In [172]:
M2$coefficients

(Intercept)          x1          x2          x3 
   3.062207   24.304260  -21.976626    3.611516

In [173]:
mean((y1-predict(M1,data.frame(x1)))^2)

[1] 0.9413354

In [174]:
mean((y2-predict(M2,data.frame(x1,x2,x3)))^2)

[1] 1.107842

---

## 시도 2